In [28]:
import torch
import pandas as pd

from tqdm.auto import tqdm
from llama2 import load_model

torch.set_grad_enabled(False)

In [2]:
model = load_model(device="cuda")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/workspace/mamba-envs/mats/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:389: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/workspace/mamba-envs/mats/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:394: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [43]:
df = pd.read_csv("datasets/cities.csv")
df.head()

,statement,label,city,country,correct_country
0,The city of Krasnodar is in Russia.,1,Krasnodar,Russia,Russia
1,The city of Krasnodar is in South Africa.,0,Krasnodar,South Africa,Russia
2,The city of Lodz is in Poland.,1,Lodz,Poland,Poland
3,The city of Lodz is in the Dominican Republic.,0,Lodz,the Dominican Republic,Poland
4,The city of Maracay is in Venezuela.,1,Maracay,Venezuela,Venezuela


In [40]:
# Grabs resid_post for all layers at the last two tokens in each statement
all_resids = []
for prompt in tqdm(df.statement):
    _, cache = model.run_with_cache(prompt, names_filter=lambda name: "resid_post" in name, return_type="loss")
    resid_stack = []
    for i in range(model.cfg.n_layers):
        resid_stack.append(cache[f"blocks.{i}.hook_resid_post"][0, -2:])
    resid_stack = torch.stack(resid_stack).cpu()
    all_resids.append(resid_stack)
all_resids = torch.stack(all_resids).cpu()

  0%|          | 0/1496 [00:00<?, ?it/s]

In [47]:
# shape: (statement layer pos d_model)
all_resids.shape

torch.Size([1496, 32, 2, 4096])

In [44]:
torch.save(all_resids, "activations/llama2-7b_cities.pt")